# very basic threads
The following shows threading


##  Synchronisation objects
POSIX - Portable operating system

 If another transaction B currently has an exclusive lock on any object matching those conditions, A must wait until B releases its lock before it is allowed to make its query.

Looking at semaphore, the analogy is that it controls the number of threads access

Semaphores are nothing more complicated than a shared counter. 
When you call down() the counter is decreased - and it will block if it cannot. If you call up() the count is incremented. (And anything blocking will be released). 
However, until that 'block' occurs, the threads can - and will - execute in an undefined order, that you should assume is random. (It isn't entirely, but relying on any particular sequence will create race conditions).
It's not complicated, but one of the things that can trip you up is buffering - print statements in threads may get buffered, so appear to arrive in strange order.

## mutex
mutex mutual exclusion  concurrency control, which is instituted for the purpose of preventing race conditions. It is the requirement that one thread of execution never enters a critical section while a concurrent thread of execution is already accessing said critical section. Python locks in p3 use lock

In [11]:
from threading import Thread
import time

def test_function(*args):
    print("From thread", ''.join(args))
    time.sleep(5)

th1 = Thread(target= test_function, args='th 1')
th2 = Thread(target= test_function, args='th 2')

th1.start()
th2.start()

print("from main")

th1.join()
th2.join()

From thread th 1
From thread th 2
from main


Python Global Interpreter Lock
Learn what Global Interpreter Lock is, how it works, and why you should use it.
A global interpreter lock (GIL) is a mechanism to apply a global lock on an interpreter. It is used in computer-language interpreters to synchronize and manage the execution of threads so that only one native thread (scheduled by the operating system) can execute at a time.
In a scenario where you have multiple threads, what can happen is that both the thread might try to acquire the memory at the same time, and as a result of which they would overwrite the data in the memory. Hence, arises a need to have a mechanism that could help prevent this phenomenon.
Some popular interpreters that have GIL are CPython and Ruby MRI. As most of you would know that Python is an interpreted language, it has various distributions like CPython, Jython, IronPython. Out of these, GIL is supported only in CPython, and it is also the most widely used implementation of Python. CPython has been developed in both C and Python language primarily to support and work with applications that have a lot of C language underneath the hood.
Even if your processor has multiple cores, a global interpreter will allow only one thread to be executed at a time. This is because, when a thread starts running, it acquires the global interpreter lock. When it waits for any I/O operation ( reading/writing data from/to disk ) or a CPU bound operation ( vector/matrix multiplication ), it releases the lock so that other threads of that process can run. Hence, it prevents you from running the other threads at the same time.

## Python multiprocessing Process class
Python multiprocessing Process class is an abstraction that sets up another Python process, provides it to run code and a way for the parent application to control execution. There are two important functions that belongs to the Process class - start() and join() function. At first, we need to write a function, that will be run by the process. Then, we need to instantiate a process object. If we create a process object, nothing will happen until we tell it to start processing via start() function. Then, the process will run and return its result. After that we tell the process to complete via join() function. Without join() function call, process will remain idle and won’t terminate. So if you create many processes and don’t terminate them, you may face scarcity of resources. Then you may need to kill them manually. One important thing is, if you want to pass any argument through the process you need to use args keyword argument. 



## Python multiprocessing Queue class
You have basic knowledge about computer data-structure, you probably know about Queue. Python Multiprocessing modules provides Queue class that is exactly a First-In-First-Out data structure. They can store any pickle Python object (though simple ones are best) and are extremely useful for sharing data between processes. Queues are specially useful when passed as a parameter to a Process’ target function to enable the Process to consume data. By using put() function we can insert data to then queue and using get() we can get items from queues. See the following code for a quick example.


## Concurrent.futures
The `concurrent.futures` module is part of the standard library which provides a high level API for launching async tasks. We will discuss and go through code samples for the common usages of this module.

Executors
This module features the `Executor` class which is an abstract class and it can not be used directly. However it has two very useful concrete subclasses – `ThreadPoolExecutor` and `ProcessPoolExecutor`. As their names suggest, one uses multi threading and the other one uses multi-processing. In both case, we get a pool of threads or processes and we can submit tasks to this pool. The pool would assign tasks to the available resources (threads or processes) and schedule them to run.

In [7]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep

def return_after_5_secs(message):
    sleep(5)
    return message

pool = ThreadPoolExecutor(3)

future = pool.submit(return_after_5_secs, ("hello"))
print(future.done())
sleep(5)
print(future.done())
print(future.result())

False
False
hello


We first construct a ThreadPoolExecutor with the number of threads we want in the pool. 

By default the number is 5 but we chose to use 3 as an example

\Then we submitted a task to the thread pool executor which waits 5 seconds before returning the message it gets as it’s first argument. When we `submit()` a task, we get back a `Future`. 

The `Future` object has a method – `done()` which tells us if the future has resolved, that is a value has been set for that particular future object. When a task finishes (returns a value or is interrupted by an exception), the thread pool executor sets the value to the future object.

In our example, the task doesn’t complete until 5 seconds, so the first call to `done()` will return `False`. We take a really short nap for 5 secs and then it’s done. We can get the result of the future by calling the `result()` method on it.